In [569]:
import torch
import json

In [599]:
index_label = {
    0: 'outside',
    1: 'nonsense',
    2: 'ileocecal',
    3: 'bbps0',
    4: 'bbps1',
    5: 'bbps2',
    6: 'bbps3',
}

In [600]:
d = json.load(
    open("/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/predicts/HYL_Clip/predict_result_MuLModel_best_cls4Acc_epoch=039_label_cleansing_acc_thresh=0.9628.ckpt.json", 'r'))
with open("/mnt/data4/cwy/Datasets/VideoHYL/pred_out.txt",'w') as f:
    for k, v in d.items():
        print("HYL_Clip/"+k,file=f,end=' ')
        res = []
        if v[0] == 1: res.append(4)
        if v[1] == 1: res.append(5)
        if v[2] == 1: res.append(6)
        if v[3] == 1: res.append(0)
        if v[4] == 1: res.append(1)
        if v[5] == 1: res.append(2)
        if v[6] == 1: res.append(3)
        sorted(res,reverse=False)
        for it in res:
            print(it,file=f,end=' ')
        f.write('\n')

In [601]:
gt_path = '/mnt/data4/cwy/Datasets/VideoHYL/gt_out.txt'
pred_path = '/mnt/data4/cwy/Datasets/VideoHYL/pred_out.txt'

In [602]:
def read_from_imagenet(txt_path: str):
    name_all = []
    label_all = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        # 0位置是图片名
        line = line.strip()
        name = line.split(' ')[0]
        name_all.append(name)
        try:
            labels = [int(x)-1 for x in line.split(' ')[1:]]
        except ValueError:
            print(line)
            raise
        label_list = [0, 0, 0, 0, 0, 0, 0]
        for label in labels:
            label_list[label] = 1
        # 把后面4个bbps标签移动到最前面
        label_list = label_list[3:] + label_list[:3]
        label_all.append(label_list)

    return name_all, torch.tensor(label_all)

In [603]:
gt_names, gt_labels = read_from_imagenet(gt_path)
pred_names, pred_labels = read_from_imagenet(pred_path)
# gt_labels与pred_labels的每一行是一个长度为7的标签向量，分别对应label_dict的每个类别

In [604]:
pred_ori=pred_labels.clone().detach()

In [605]:
outlier_json_path = '/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/outliers/outlier_result.json'

In [606]:
outliers = json.load(open(outlier_json_path))['HYL_Clip']
outliers

{'0': {'0': [[122, 124],
   [172, 174],
   [178, 178],
   [430, 430],
   [446, 449],
   [451, 457],
   [460, 460],
   [466, 487],
   [497, 505],
   [507, 507],
   [656, 679],
   [1758, 1786],
   [1788, 1788],
   [1791, 1804],
   [1806, 1806],
   [1808, 1816],
   [1818, 1831],
   [1833, 1836],
   [1904, 1953],
   [1955, 2003],
   [3950, 3988],
   [3990, 4008],
   [4010, 4010],
   [4012, 4021],
   [4023, 4026],
   [4028, 4043],
   [4045, 4046],
   [4115, 4135],
   [4137, 4141],
   [4144, 4145],
   [4147, 4148],
   [4533, 4541],
   [4543, 4543],
   [4546, 4576],
   [4646, 4678],
   [4680, 4726],
   [4971, 4977],
   [4979, 4983]],
  '1': [[125, 171],
   [175, 177],
   [431, 445],
   [450, 450],
   [458, 459],
   [461, 465],
   [488, 496],
   [506, 506],
   [508, 509],
   [654, 655],
   [680, 680],
   [1757, 1757],
   [1787, 1787],
   [1789, 1790],
   [1805, 1805],
   [1807, 1807],
   [1817, 1817],
   [1832, 1832],
   [1837, 1837],
   [1903, 1903],
   [1954, 1954],
   [2004, 2004],
   [3320

In [607]:
outside_outliers_101 = outliers['0']['0']
outside_outliers_010 = outliers['0']['1']
nonsense_outliers_101 = outliers['1']['0']
nonsense_outliers_010 = outliers['1']['1']

In [608]:
def merge_outside_outliers(idx_101,idx_010):
    idx_ab=[]
    idx_ab.extend(idx_101)
    idx_ab.extend(idx_010)
    idx_ab.sort(key=lambda x:x[0])
    idx_merged = [idx_ab.pop(0)]
    for it in idx_ab:
        if it[0]==idx_merged[-1][1]+1:
            idx_merged[-1][1]=it[1]
        else:
            idx_merged.append(it)
    return idx_merged 

In [609]:
outside_outliers = merge_outside_outliers(outside_outliers_101,outside_outliers_010)
outside_outliers

[[122, 178],
 [430, 509],
 [654, 680],
 [1757, 1837],
 [1903, 2004],
 [3320, 3320],
 [3948, 4047],
 [4114, 4149],
 [4315, 4315],
 [4430, 4430],
 [4532, 4577],
 [4645, 4727],
 [4970, 4984],
 [5069, 5073],
 [5206, 5206]]

In [610]:
nonsense_outliers = nonsense_outliers_101
nonsense_outliers

[[594, 594],
 [597, 597],
 [611, 611],
 [629, 631],
 [633, 633],
 [676, 676],
 [678, 680],
 [684, 684],
 [686, 687],
 [694, 695],
 [698, 700],
 [709, 711],
 [713, 713],
 [715, 716],
 [730, 731],
 [741, 741],
 [784, 784],
 [786, 788],
 [864, 864],
 [866, 868],
 [900, 902],
 [975, 975],
 [996, 997],
 [1006, 1006],
 [1066, 1066],
 [1069, 1071],
 [1142, 1142],
 [1158, 1159],
 [1224, 1224],
 [1240, 1242],
 [1266, 1267],
 [1269, 1269],
 [1347, 1347],
 [1350, 1350],
 [1352, 1354],
 [1377, 1379],
 [1382, 1382],
 [1384, 1384],
 [1386, 1386],
 [1389, 1389],
 [1523, 1524],
 [1539, 1539],
 [1542, 1542],
 [1584, 1584],
 [1586, 1588],
 [1590, 1591],
 [1627, 1627],
 [1784, 1785],
 [1908, 1909],
 [1919, 1919],
 [1941, 1941],
 [1950, 1951],
 [1954, 1954],
 [2044, 2046],
 [2050, 2051],
 [2053, 2054],
 [2190, 2190],
 [2199, 2199],
 [2355, 2355],
 [2481, 2483],
 [2547, 2549],
 [2559, 2559],
 [3314, 3315],
 [3408, 3408],
 [3541, 3543],
 [3572, 3574],
 [3589, 3589],
 [3629, 3629],
 [3709, 3709],
 [3837, 383

In [611]:
def fix_outliers_idx(pred_origin, outlier_list, val_func):
    pred_fix = pred_origin.clone().detach()
    for item in outlier_list:
        a, b = item
        for now_i in range(a,b+1):
            pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))
    return pred_fix

In [612]:
def val_reverse_outside(pred,idx_item,now_i):
    a, b = idx_item
    if pred[a][0].item()==0:
        if pred[now_i][0].item()==1 or pred[now_i][1].item()==1:
            print(f'Fix {idx_item}: {now_i} not change')
            return pred[now_i].clone().detach()
        else:
            print(f'Fix {idx_item}: {now_i}=[1, 0, 0, 0, 0, 0, 0]')
            return [1, 0, 0, 0, 0, 0, 0]
    elif pred[a][0].item()==1:
        if pred[now_i][0].item()==0:
            print(f'Fix {idx_item}: {now_i} not change')
            return pred[now_i].clone().detach()
        else:
            print(f'Fix {idx_item}: {now_i}=[0, 1, 0, 0, 0, 0, 0]')
            return [0, 1, 0, 0, 0, 0, 0]
    else:
        raise ValueError

In [613]:
def val_reverse_nonsense_101(pred,idx_item,now_i):
    if pred[now_i][1].item()==0:
        print(f'Fix {idx_item}: {now_i}=[0, 1, 0, 0, 0, 0, 0]')
        return [0, 1, 0, 0, 0, 0, 0]
    elif pred[now_i][1].item()==1:
        print(f'Fix {idx_item}: {now_i} not change')
        return pred[now_i].clone().detach()
    else:
        raise ValueError

In [614]:
pred_fix = pred_labels.clone().detach()

In [615]:
pred_fix = fix_outliers_idx(pred_fix, outside_outliers, val_reverse_outside)

Fix [122, 178]: 122 not change
Fix [122, 178]: 123 not change
Fix [122, 178]: 124 not change
Fix [122, 178]: 125 not change
Fix [122, 178]: 126 not change
Fix [122, 178]: 127 not change
Fix [122, 178]: 128 not change
Fix [122, 178]: 129 not change
Fix [122, 178]: 130 not change
Fix [122, 178]: 131 not change
Fix [122, 178]: 132 not change
Fix [122, 178]: 133 not change
Fix [122, 178]: 134 not change
Fix [122, 178]: 135 not change
Fix [122, 178]: 136 not change
Fix [122, 178]: 137 not change
Fix [122, 178]: 138 not change
Fix [122, 178]: 139 not change
Fix [122, 178]: 140 not change
Fix [122, 178]: 141 not change
Fix [122, 178]: 142 not change
Fix [122, 178]: 143 not change
Fix [122, 178]: 144 not change
Fix [122, 178]: 145 not change
Fix [122, 178]: 146 not change
Fix [122, 178]: 147 not change
Fix [122, 178]: 148 not change
Fix [122, 178]: 149 not change
Fix [122, 178]: 150 not change
Fix [122, 178]: 151 not change
Fix [122, 178]: 152 not change
Fix [122, 178]: 153 not change
Fix [122

/tmp/ipykernel_3822/1748927244.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))


In [616]:
pred_fix = fix_outliers_idx(pred_fix, nonsense_outliers, val_reverse_nonsense_101)

Fix [594, 594]: 594=[0, 1, 0, 0, 0, 0, 0]
Fix [597, 597]: 597=[0, 1, 0, 0, 0, 0, 0]
Fix [611, 611]: 611=[0, 1, 0, 0, 0, 0, 0]
Fix [629, 631]: 629=[0, 1, 0, 0, 0, 0, 0]
Fix [629, 631]: 630=[0, 1, 0, 0, 0, 0, 0]
Fix [629, 631]: 631=[0, 1, 0, 0, 0, 0, 0]
Fix [633, 633]: 633=[0, 1, 0, 0, 0, 0, 0]
Fix [676, 676]: 676=[0, 1, 0, 0, 0, 0, 0]
Fix [678, 680]: 678=[0, 1, 0, 0, 0, 0, 0]
Fix [678, 680]: 679=[0, 1, 0, 0, 0, 0, 0]
Fix [678, 680]: 680 not change
Fix [684, 684]: 684=[0, 1, 0, 0, 0, 0, 0]
Fix [686, 687]: 686=[0, 1, 0, 0, 0, 0, 0]
Fix [686, 687]: 687=[0, 1, 0, 0, 0, 0, 0]
Fix [694, 695]: 694=[0, 1, 0, 0, 0, 0, 0]
Fix [694, 695]: 695=[0, 1, 0, 0, 0, 0, 0]
Fix [698, 700]: 698=[0, 1, 0, 0, 0, 0, 0]
Fix [698, 700]: 699=[0, 1, 0, 0, 0, 0, 0]
Fix [698, 700]: 700=[0, 1, 0, 0, 0, 0, 0]
Fix [709, 711]: 709=[0, 1, 0, 0, 0, 0, 0]
Fix [709, 711]: 710=[0, 1, 0, 0, 0, 0, 0]
Fix [709, 711]: 711=[0, 1, 0, 0, 0, 0, 0]
Fix [713, 713]: 713=[0, 1, 0, 0, 0, 0, 0]
Fix [715, 716]: 715=[0, 1, 0, 0, 0, 0, 0]
Fix

/tmp/ipykernel_3822/1748927244.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))


In [617]:
pred_labels = pred_fix.clone().detach()

In [618]:
confuse_matrix = dict()
for i in range(3):
    confuse_matrix[f'label_{index_label[i]}_TP'] = 0.
    confuse_matrix[f'label_{index_label[i]}_FP'] = 0.
    confuse_matrix[f'label_{index_label[i]}_FN'] = 0.
    confuse_matrix[f'label_{index_label[i]}_TN'] = 0.
for i in range(0, 4):  # i: predict
    for j in range(0, 4):  # j: gt
        confuse_matrix[f'label_cleansing_pred_{index_label[i + 3]}_gt_{index_label[j + 3]}'] = 0.

In [619]:
# 计算test_acc
mean_acc = float(torch.eq(pred_labels, gt_labels).float().mean().cpu())

In [620]:
in_out_labels = pred_labels[:, 0]
# 体内外标签: BoolTensor[B]
# outside时为True
label_in_out_pred = torch.gt(in_out_labels, 0)
# 体内外gt: BoolTensor[B]
label_in_out_gt = torch.gt(gt_labels[:, 0], 0)
confuse_matrix[f'label_{index_label[0]}_TP'] += float((label_in_out_pred & label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_FP'] += float((label_in_out_pred & ~label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_FN'] += float((~label_in_out_pred & label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_TN'] += float((~label_in_out_pred & ~label_in_out_gt).float().sum().cpu())


In [621]:
nonsense_logit = pred_labels[:, 1]
# 坏帧标签: BoolTensor[B]
# nonsense时为True
label_nonsense_pred = torch.gt(nonsense_logit, 0)
# 帧质量gt: BoolTensor[B]
# pred或gt是outside时不计入总数
label_nonsense_gt = torch.gt(gt_labels[:, 1], 0)
flag = ~label_in_out_pred & ~label_in_out_gt
confuse_matrix[f'label_{index_label[1]}_TP'] += float(
    (flag & label_nonsense_pred & label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_FP'] += float(
    (flag & label_nonsense_pred & ~label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_FN'] += float(
    (flag & ~label_nonsense_pred & label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_TN'] += float(
    (flag & ~label_nonsense_pred & ~label_nonsense_gt).float().sum().cpu())


In [622]:
ileo_logit = pred_labels[:, 2]
# 回盲部标签: BoolTensor[B]
label_ileo_pred = torch.gt(ileo_logit, 0)
# 回盲部gt: BoolTensor[B]
label_ileo_gt = torch.gt(gt_labels[:, 2], 0)
flag = ~label_in_out_pred & ~label_in_out_gt & ~label_nonsense_pred & ~label_nonsense_gt
confuse_matrix[f'label_{index_label[2]}_TP'] += float((flag & label_ileo_pred & label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_FP'] += float((flag & label_ileo_pred & ~label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_FN'] += float((flag & ~label_ileo_pred & label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_TN'] += float((flag & ~label_ileo_pred & ~label_ileo_gt).float().sum().cpu())


In [623]:
# 清洁度logit: FloatTensor[B, 4]
cls_logit = pred_labels[:, 3:]
# 清洁度label: IntTensor[B] (取预测值最大的，但会被outside标签抑制)
label_cls_pred = torch.argmax(cls_logit, dim=-1)
# 清洁度gt: IntTensor[B]
label_cls_gt = torch.argmax(gt_labels[:, 3:], dim=-1)
flag = ~label_in_out_pred & ~label_in_out_gt & ~label_nonsense_pred & ~label_nonsense_gt
for i in range(0, 4):  # i: predict
    for j in range(0, 4):  # j: gt
        confuse_matrix[f'label_cleansing_pred_{index_label[i + 3]}_gt_{index_label[j + 3]}'] += float((flag & torch.eq(label_cls_pred, i) & torch.eq(label_cls_gt, j)).float().sum().cpu())  # flag用于清洁度标签抑制


In [624]:
confuse_matrix

{'label_outside_TP': 517.0,
 'label_outside_FP': 3.0,
 'label_outside_FN': 0.0,
 'label_outside_TN': 4814.0,
 'label_nonsense_TP': 1400.0,
 'label_nonsense_FP': 41.0,
 'label_nonsense_FN': 40.0,
 'label_nonsense_TN': 3333.0,
 'label_ileocecal_TP': 119.0,
 'label_ileocecal_FP': 3.0,
 'label_ileocecal_FN': 5.0,
 'label_ileocecal_TN': 3206.0,
 'label_cleansing_pred_bbps0_gt_bbps0': 31.0,
 'label_cleansing_pred_bbps0_gt_bbps1': 1.0,
 'label_cleansing_pred_bbps0_gt_bbps2': 62.0,
 'label_cleansing_pred_bbps0_gt_bbps3': 3.0,
 'label_cleansing_pred_bbps1_gt_bbps0': 0.0,
 'label_cleansing_pred_bbps1_gt_bbps1': 170.0,
 'label_cleansing_pred_bbps1_gt_bbps2': 9.0,
 'label_cleansing_pred_bbps1_gt_bbps3': 2.0,
 'label_cleansing_pred_bbps2_gt_bbps0': 0.0,
 'label_cleansing_pred_bbps2_gt_bbps1': 15.0,
 'label_cleansing_pred_bbps2_gt_bbps2': 1262.0,
 'label_cleansing_pred_bbps2_gt_bbps3': 88.0,
 'label_cleansing_pred_bbps3_gt_bbps0': 0.0,
 'label_cleansing_pred_bbps3_gt_bbps1': 0.0,
 'label_cleansing_p

In [625]:
metrics = dict()

# 体内外
TP: float = confuse_matrix[f'label_{index_label[0]}_TP']
FP: float = confuse_matrix[f'label_{index_label[0]}_FP']
FN: float = confuse_matrix[f'label_{index_label[0]}_FN']
TN: float = confuse_matrix[f'label_{index_label[0]}_TN']
metrics[f'label_{index_label[0]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 帧质量
TP: float = confuse_matrix[f'label_{index_label[1]}_TP']
FP: float = confuse_matrix[f'label_{index_label[1]}_FP']
FN: float = confuse_matrix[f'label_{index_label[1]}_FN']
TN: float = confuse_matrix[f'label_{index_label[1]}_TN']
metrics[f'label_{index_label[1]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 回盲部
TP: float = confuse_matrix[f'label_{index_label[2]}_TP']
FP: float = confuse_matrix[f'label_{index_label[2]}_FP']
FN: float = confuse_matrix[f'label_{index_label[2]}_FN']
TN: float = confuse_matrix[f'label_{index_label[2]}_TN']

metrics[f'label_{index_label[2]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 四分清洁度准确率
total: float = 0.
correct: float = 0.
for i in range(3, 7):  # i: predict
    for j in range(3, 7):  # j: gt
        tmp = confuse_matrix[f'label_cleansing_pred_{index_label[i]}_gt_{index_label[j]}']
        if i == j:
            correct += tmp
        total += tmp
metrics[f'label_cleansing_acc'] = correct / total if total > 0. else 0.

In [626]:
mean_acc

0.9831271171569824

In [627]:
metrics

{'label_outside_acc': 0.999437570303712,
 'label_nonsense_acc': 0.983174075612796,
 'label_ileocecal_acc': 0.9975997599759976,
 'label_cleansing_acc': 0.931893189318932}

In [106]:
for i in range(pred_labels.size()[0]):
    if torch.not_equal(pred_labels[i],gt_labels[i]).any():
        print(f"{i}:{pred_labels[i].numpy()} != {gt_labels[i].numpy()}")

497:[1 0 0 0 0 0 0] != [0 1 0 0 0 0 0]
498:[1 0 0 0 0 0 0] != [0 1 0 0 0 0 0]
499:[1 0 0 0 0 0 0] != [0 1 0 0 0 0 0]
668:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
671:[0 0 0 0 0 0 1] != [0 0 0 0 0 1 0]
1050:[0 0 0 0 1 0 0] != [0 0 0 0 0 1 0]
1061:[0 0 0 0 0 1 0] != [0 0 0 0 0 0 1]
1069:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1070:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1071:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1142:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1158:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1159:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1224:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1236:[0 0 0 0 1 0 0] != [0 0 0 0 0 0 1]
1241:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1242:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1266:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1267:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1269:[0 1 0 0 0 0 0] != [0 0 0 0 0 1 0]
1524:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
1751:[0 0 0 0 0 0 1] != [0 0 0 0 0 1 0]
1759:[0 0 0 0 0 1 0] != [0 1 0 0 0 0 0]
1760:[0 0 0 0 0 0 1] != [0 0 0 0 0 1 0]
1784:[0 1 0 0 0 0 0] != [0 0 0 0 0 1 0]
1791:

In [107]:
import DrawLabelColorBlock
import os
import os.path as osp
import shutil

In [108]:
if osp.exists('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred'):
    shutil.rmtree('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred')
os.makedirs('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred')

In [109]:
for i,pd in enumerate(pred_labels):
    ori_img_path = osp.join('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames',pred_names[i])
    out_img_path = osp.join('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred',pred_names[i])
    pd_np=pd.numpy()
    pd_dict={'outside': False,
           'nonsense': False,
           'ileocecal': False,
           'bbps': -1}
    if pd_np[0]==1:
        pd_dict['outside']=True
    if pd_np[1]==1:
        pd_dict['nonsense']=True
    if pd_np[2]==1:
        pd_dict['ileocecal']=True
    if pd_np[3]==1:
        pd_dict['bbps']=0
    elif pd_np[4]==1:
        pd_dict['bbps']=1
    elif pd_np[5]==1:
        pd_dict['bbps']=2
    elif pd_np[6]==1:
        pd_dict['bbps']=3
    DrawLabelColorBlock.DrawLabelColorBlockOnFrame(ori_img_path,out_img_path,**pd_dict)

In [110]:

# 把帧拼成视频
img_path = '/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred/HYL_Clip/'
img_list = os.listdir(img_path)
img_list.sort(key=lambda x: int(x.split('.')[0]))
img_list

['000000.png',
 '000002.png',
 '000004.png',
 '000006.png',
 '000008.png',
 '000010.png',
 '000012.png',
 '000014.png',
 '000016.png',
 '000018.png',
 '000020.png',
 '000022.png',
 '000024.png',
 '000026.png',
 '000028.png',
 '000030.png',
 '000032.png',
 '000034.png',
 '000036.png',
 '000038.png',
 '000040.png',
 '000042.png',
 '000044.png',
 '000046.png',
 '000048.png',
 '000050.png',
 '000052.png',
 '000054.png',
 '000056.png',
 '000058.png',
 '000060.png',
 '000062.png',
 '000064.png',
 '000066.png',
 '000068.png',
 '000070.png',
 '000072.png',
 '000074.png',
 '000076.png',
 '000078.png',
 '000080.png',
 '000082.png',
 '000084.png',
 '000086.png',
 '000088.png',
 '000090.png',
 '000092.png',
 '000094.png',
 '000096.png',
 '000098.png',
 '000100.png',
 '000102.png',
 '000104.png',
 '000106.png',
 '000108.png',
 '000110.png',
 '000112.png',
 '000114.png',
 '000116.png',
 '000118.png',
 '000120.png',
 '000122.png',
 '000124.png',
 '000126.png',
 '000128.png',
 '000130.png',
 '000132.p

In [111]:
import cv2
# cv2
img = cv2.imread(osp.join(img_path, img_list[0]))
height, width, layers = img.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
os.makedirs('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/videos_with_pred', exist_ok=True)
video = cv2.VideoWriter('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/videos_with_pred/HYL_Clip.mp4', fourcc, 12.5, (width, height))

for img_name in img_list:
    img = cv2.imread(osp.join(img_path, img_name))
    video.write(img)
    
cv2.destroyAllWindows()
video.release()



